# Create winterYear snowDynamics products
- 1st run: with implausible snow filter (7 day buffer): is_7 (default)
- 2nd run: with implausible snow filter (30 day buffer): is_30
- 2nd run: without implausible snow filter: is_none

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import xarray as xr
import os
import gc

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing, removing memory_limit is important
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59635,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:59670,Total threads: 10
Dashboard: http://127.0.0.1:59677/status,Memory: 0 B
Nanny: tcp://127.0.0.1:59638,


2025-02-18 16:21:59,630 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 69ed3d5ff65fec418bed6463799e938a initialized by task ('rechunk-merge-rechunk-transfer-58b9066029f15b4b1b500af50c267fa6', 0, 0, 0, 31, 0, 0) executed on worker tcp://127.0.0.1:59689
2025-02-18 16:22:26,200 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 69ed3d5ff65fec418bed6463799e938a deactivated due to stimulus 'task-finished-1739913746.0968068'
2025-02-18 16:23:49,783 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 69ed3d5ff65fec418bed6463799e938a initialized by task ('rechunk-merge-rechunk-transfer-58b9066029f15b4b1b500af50c267fa6', 0, 0, 0, 31, 0, 0) executed on worker tcp://127.0.0.1:59671
2025-02-18 16:24:00,966 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 69ed3d5ff65fec418bed6463799e938a deactivated due to stimulus 'task-finished-1739913840.7329762'
2025-02-18 16:26:06,572 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 69ed3d5ff65fec418bed646379

In [3]:
# Calculated from IMS (4 SnowDynamics_IMS)
sstart_first = 248
send_last = 166
smin_avg = 205

In [5]:
wys = 2023
wye = 2024
wy = str(wys)[2:] + str(wye)[2:]

fmask = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\High Resolution Snow Dynamics\Fmask_V1\AA\Alaska Snow Depth\1 HLS Fmask from STAC\fmask_nc' # Location of initial Fmask tiles downloaded from STAC
path = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\High Resolution Snow Dynamics\Fmask_V1\AA\Alaska Snow Depth\3 HLS snowDynamics\is_7\winterYear' # Path to save outputs

output_ID = 'winterYear' + wy + '_AlaskaSite'

thresh = 2 # During cleaning, require this many consecutive snow/non-snow observations

In [8]:
%%time

# Load annual fask cubes
fmask1 = pputil.uploadNC(os.path.join(fmask, 'fmask_AlaskaSite_' + str(wys) + '.nc'), form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
fmask2 = pputil.uploadNC(os.path.join(fmask, 'fmask_AlaskaSite_' + str(wye) + '.nc'), form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
print('Loaded annual Fmask cubes (' + str(wys) + ': ' + str(len(fmask1)) +  ' time-steps, ' + str(wye) + ': ' + str(len(fmask2)) + ' time-steps).')

# Convert to snow cube
snowCube = snow_util.annualFmask2SnowCube(fmask1, fmask2, yrs = 2, verbose = False)
print('Converted to 2-year snowCube, including merging same-day & removing 99+% unclear time-steps (' + str(len(snowCube)) + ' time-steps).')
    
# Get observation frequency
# clearObsPerDay = snow_util.clearObservationsPerDay_WinterYear(snowCube, windows = ['winterYear', 'snowSeason', 'snowFall', 'snowPeak', 'snowMelt'], 
#                                                               smin_doy = int(tile_stats.loc[tile, 'smin_avg']),
#                                                               smax_doy = int(tile_stats.loc[tile, 'smax_avg']), 
#                                                               sstart_doy = int(tile_stats.loc[tile, 'sstart_avg']),
#                                                               send_doy = int(tile_stats.loc[tile, 'send_avg']), 
#                                                               verbose = False)
# #clearObsPerDay = pputil.loadXR(clearObsPerDay)
# print('Output clearObsPerDay for temporal windows ' + str(tuple([str.split('_')[0] for str in list(clearObsPerDay.keys())])) + '.')
    
# Identify implausible snow period
implausible_snow = [send_last + 7, sstart_first - 7] # Week buffer
#implausible_snow = [send_last + 30, sstart_first - 30] # Month buffer
#implausible_snow = [] # No implausible snow
            
date1 = dt.datetime(wys, 12, 31) + dt.timedelta(implausible_snow[0])
date1 = str(date1.month) + '/' + str(date1.day)
    
date2 = dt.datetime(wys, 12, 31) + dt.timedelta(implausible_snow[1])
date2 = str(date2.month) + '/' + str(date2.day)
            
print('Implausible snow period identified as ' + date1 + ' (doy ' + str(implausible_snow[0]) + ') to ' + date2 + ' (doy ' + str(implausible_snow[1]) + ').')
    
# Clean snow cube
snowCube = snow_util.cleanSnowCube(snowCube,  form = 'binary', temporal = 'gaps', implausible_snow = implausible_snow, thresh = thresh, verbose = False)
print('Cleaned snowCube, including setting non-snow during implausible snow period & removing snow/non-snow periods below ' + str(thresh) + ' consecutive observations.')
    
# Convert to daily snow cube
snowCube = snow_util.dailySnowCube(snowCube, doy = smin_avg, verbose = False)
        
first_day = str(snowCube['time'][0].values)[0:10]
last_day = str(snowCube['time'][-1].values)[0:10]
        
print('Converted to winterYear daily snowCube (' + first_day + ' to ' + last_day + ', ' + str(len(snowCube)) + ' time-steps), including linear intepolation to fill gaps.')

# # Load into memory
# snowCube = pputil.loadXR(snowCube)
# print('Loaded snowCube into memory.')

# Calculate snow dynamis
snowDynamics = snow_util.dailySnowCube2SnowDynamics(snowCube, products = ['start', 'end', 'length', 'periods', 'status'], uncertainty = True, verbose = False)
print('Output winterYear snowDynamics products ' + str(tuple([str[5:] for str in list(snowDynamics.keys())])) + '.')
    
# Merge clearObsPerDay and snowDynamics into dataset to save
#snowDynamics = xr.merge([snowDynamics, clearObsPerDay])
snowDynamics = snowDynamics.expand_dims(winterYear = xr.Variable('winterYear', [str(wys) + '-' + str(wye)])) # Winter year as time dimension
#print('Merged winterYear snowDynamics and clearObsPerDay into single dataset (' + str(len(snowDynamics) + ' variables).'))
    
# Final cleaning and save dataset to TIFs
snowDynamics = snowDynamics.reset_coords(names = ['band'], drop = True) # Remove unnessisary 'band' coordinate

variables = list(snowDynamics.keys()) # Name of all data variables as list 

# Load into memory
snowDynamics = pputil.loadXR(snowDynamics)
print('Loaded snowDynamics into memory.')

for variable in variables:

    da = snowDynamics[variable].astype('float32') # Single snow dynamic stat from dataset

    da.rio.write_nodata(-999, encoded = True, inplace = True) # Ensured NaN NoData values are saved
    da.rio.to_raster(os.path.join(path, str(wy), 'HLS_Fmask_' + variable + '_' + output_ID + '.tif'))        

    print('Product ' + str(variables.index(variable) + 1) + ' of ' + str(len(variables)) + ' saved: HLS_Fmask_' + variable + '_' + output_ID + '.tif')

Loaded annual Fmask cubes (2023: 242 time-steps, 2024: 245 time-steps).
Converted to 2-year snowCube, including merging same-day & removing 99+% unclear time-steps (253 time-steps).
Implausible snow period identified as 6/21 (doy 173) to 8/28 (doy 241).
Cleaned snowCube, including setting non-snow during implausible snow period & removing snow/non-snow periods below 2 consecutive observations.
Converted to winterYear daily snowCube (2023-07-25 to 2024-07-24, 366 time-steps), including linear intepolation to fill gaps.
Output winterYear snowDynamics products ('start', 'end', 'length', 'periods', 'status', 'start_u', 'end_u', 'length_u').
Loaded snowDynamics into memory.
Product 1 of 8 saved: HLS_Fmask_snow_start_winterYear2324_AlaskaSite.tif
Product 2 of 8 saved: HLS_Fmask_snow_end_winterYear2324_AlaskaSite.tif
Product 3 of 8 saved: HLS_Fmask_snow_length_winterYear2324_AlaskaSite.tif
Product 4 of 8 saved: HLS_Fmask_snow_periods_winterYear2324_AlaskaSite.tif
Product 5 of 8 saved: HLS_Fma

In [6]:
# Memory management at end
del fmask1, fmask2, snowCube, snowDynamics # clearObsPerDay
client.run(gc.collect) # Collect dask worker garbage
#client.restart()

{'tcp://127.0.0.1:58289': 645,
 'tcp://127.0.0.1:58290': 370,
 'tcp://127.0.0.1:58291': 379,
 'tcp://127.0.0.1:58292': 480,
 'tcp://127.0.0.1:58293': 783,
 'tcp://127.0.0.1:58294': 395,
 'tcp://127.0.0.1:58315': 423,
 'tcp://127.0.0.1:58316': 437,
 'tcp://127.0.0.1:58317': 576,
 'tcp://127.0.0.1:58318': 533}